In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("movie-recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

In [3]:
ratings_file = "/Users/keon/fastcampus/data-engineering/01-spark/data/ml-25m/ratings.csv"
ratings_df = spark.read.csv(f"file:///{ratings_file}", inferSchema=True, header=True)

In [4]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [5]:
ratings_df = ratings_df.select(["userId", "movieId", "rating"])

In [6]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [7]:
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423535|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [8]:
train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
als = ALS(
    maxIter=5,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

In [11]:
model = als.fit(train_df)

In [12]:
predictions = model.transform(test_df)

In [13]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    85|   1088|   2.0| 3.6265106|
|   101|   8638|   5.0|  3.544008|
|   243|   1580|   3.0|  2.657289|
|   296|   1580|   3.0| 2.7011428|
|   321|   3175|   3.0|  3.179428|
|   321|   6620|   3.5|  3.655777|
|   368|   1580|   3.5|  3.612191|
|   385|   1088|   3.0| 2.9537084|
|   481|   1580|   4.0|  3.659029|
|   588|   1580|   2.5| 2.7396905|
|   588|   1645|   2.5| 2.5747867|
|   597|   3175|   5.0|  3.933945|
|   597|   3997|   1.0| 1.9786683|
|   613|   1088|   4.0| 3.0452945|
|   633|   1591|   5.0|  3.226123|
|   744|  44022|   3.5| 3.6344576|
|   772|   2122|   2.0| 2.0285587|
|   830|   1580|   5.0| 3.5411048|
|   833|   3175|   5.0| 3.3417945|
|   847|   1959|   2.0| 3.2581763|
+------+-------+------+----------+
only showing top 20 rows



In [16]:
predictions.select('rating', 'prediction').describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4996260|           4996260|
|   mean|3.5341314303098716|3.3965898291399603|
| stddev|1.0602933602351239|0.6386960626431551|
|    min|               0.5|        -1.5794845|
|    max|               5.0|          6.671399|
+-------+------------------+------------------+



In [17]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol='rating', predictionCol='prediction')

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print(rmse)

0.8136400105983754


In [21]:
model.recommendForAllUsers(3).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[{177209, 7.46965...|
|    31|[{177209, 3.99431...|
|    34|[{177209, 6.06151...|
|    53|[{194334, 6.95767...|
|    65|[{149484, 6.86115...|
|    78|[{177209, 7.05879...|
|    81|[{200930, 5.32819...|
|    85|[{149484, 5.79808...|
|   101|[{203086, 5.36891...|
|   108|[{149484, 5.69486...|
|   115|[{177209, 6.40233...|
|   126|[{203086, 6.58937...|
|   133|[{203086, 5.52253...|
|   137|[{203086, 5.57281...|
|   148|[{177209, 5.89362...|
|   155|[{202231, 6.02274...|
|   183|[{177209, 6.10260...|
|   193|[{177209, 5.39314...|
|   210|[{139036, 8.41384...|
|   211|[{203086, 6.66992...|
+------+--------------------+
only showing top 20 rows



In [22]:
model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     28|[{105801, 5.48301...|
|     31|[{10417, 5.051482...|
|     34|[{128562, 5.38732...|
|     53|[{96740, 5.325608...|
|     65|[{87426, 4.871016...|
|     78|[{67467, 4.594074...|
|     81|[{7629, 4.6699777...|
|     85|[{105801, 4.77481...|
|    101|[{142811, 4.89765...|
|    108|[{4243, 4.9666142...|
|    115|[{105801, 5.67604...|
|    126|[{87426, 4.70493}...|
|    133|[{119077, 5.08030...|
|    137|[{113441, 4.99710...|
|    148|[{96740, 4.183742...|
|    155|[{10417, 5.021953...|
|    183|[{87426, 5.186278...|
|    193|[{87426, 5.032519...|
|    210|[{67467, 4.752000...|
|    211|[{105801, 5.12777...|
+-------+--------------------+
only showing top 20 rows



In [23]:
from pyspark.sql.types import IntegerType

user_list = [65, 78, 81]
users_df = spark.createDataFrame(user_list, IntegerType()).toDF('userId')

users_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [27]:
user_recs = model.recommendForUserSubset(users_df, 5)

In [28]:
movies_list = user_recs.collect()[0].recommendations

In [30]:
recs_df = spark.createDataFrame(movies_list)
recs_df.show()

+-------+-----------------+
|movieId|           rating|
+-------+-----------------+
| 149484| 6.86115837097168|
| 159761|6.331261157989502|
| 139036|6.242280960083008|
| 197433|6.134408950805664|
| 203882|5.953912734985352|
+-------+-----------------+



In [31]:
movies_file = "/Users/keon/fastcampus/data-engineering/01-spark/data/ml-25m/movies.csv"
movies_df = spark.read.csv(f"file:///{movies_file}", inferSchema=True, header=True)

In [32]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [33]:
recs_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView("movies")

In [34]:
query = """
SELECT *
FROM
    movies JOIN recommendations
    ON movies.movieId = recommendations.movieId
ORDER BY
    rating desc
"""
recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+--------------+-------+-----------------+
|movieId|               title|        genres|movieId|           rating|
+-------+--------------------+--------------+-------+-----------------+
| 149484|All About My Wife...|Comedy|Romance| 149484| 6.86115837097168|
| 159761|         Loot (1970)|  Comedy|Crime| 159761|6.331261157989502|
| 139036|World Gone Wild (...| Action|Sci-Fi| 139036|6.242280960083008|
| 197433|Temporary Difficu...|         Drama| 197433|6.134408950805664|
| 203882|Dead in the Water...|        Horror| 203882|5.953912734985352|
+-------+--------------------+--------------+-------+-----------------+



In [35]:
def get_recommendations(user_id, num_recs):
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF('userId')
    user_recs_df = model.recommendForUserSubset(users_df, num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list)
    recommended_movies = spark.sql(query)
    return recommended_movies

In [36]:
recs = get_recommendations(456, 10)

In [37]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,149484,All About My Wife (2012),Comedy|Romance,149484,6.861158
1,159761,Loot (1970),Comedy|Crime,159761,6.331261
2,139036,World Gone Wild (1988),Action|Sci-Fi,139036,6.242281
3,197433,Temporary Difficulties (2018),Drama,197433,6.134409
4,203882,Dead in the Water (2006),Horror,203882,5.953913


In [38]:
spark.stop()